<font color=red size=6> $$head </font>

In [1]:
import re
import nltk
import brtoz.brmagic

## 正则表达式操作

https://docs.python.org/3/library/re.html#id1  


### 模式语法

**正则表达式中的特殊字符**:

> |字符|描述|
|----|---|
| text | 匹配普通字符 |
| . | 匹配除换行符以外的任意字符(与flags有关) |
| ^ | 零宽界定符, 匹配字符串的开始 |
| $ | 零宽界定符, 匹配字符串的结束 |
| * | 匹配前面表达式的零个或多个副本, 匹配尽可能多的副本    |
| + | 匹配前面表达式的一个或多个副本, 匹配尽可能多的副本 |
| ? | 匹配前面表达式的零个或一个副本, 匹配尽可能多的副本 |
| *? | 匹配前面表达式的零个或多个副本, 匹配尽可能少的副本    |
| +? | 匹配前面表达式的一个或多个副本, 匹配尽可能少的副本 |
| ?? | 匹配前面表达式的零个或一个副本, 匹配尽可能少的副本 |
| {m} | 准确匹配前面表达式的m个副本 |
| {m,n} | 匹配前面表达式的m至n个副本, 匹配尽可能多的副本; m0=0,n0=inf;{}?表示懒惰模式 |
| [...] | 匹配字符集中的一个字符(可使用`-`); 特殊字符在字符集中会失去特殊性 |
| [^...] | 字符集的补集 |
| A &#124; B | `re`中的管道符, 不含管道(串流)的含义, 而只是单纯的表示选择(并列) |
| (...) | 设置子模式并保存子匹配项, 子项可通过group方法获取 |
| (?aiLmsux) | 可修改re规则的flags, 例如是否忽略大小写等 |
| (?:...) | 设置子模式, 但丢弃子项 |
| (?P<name\>...) | 为子模式指定分组名 |
| (?P=name) | 通过组名反向引用子模式 |
| (?#...) | 一个注释, `()`中的内容将被忽略 |
| (?=...) | 后置肯定界定符, 只有在括号中的模式匹配时, 才匹配前面的表达式 |
| (?!...) | 后置否定界定符, 只有在括号中的模式不匹配时, 才匹配前面的表达式 |
| (?<=...) | 前置肯定界定符, 只有在括号中的模式匹配时, 才匹配后面的表达式 |
| (?<!...) | 前置否定界定符, 只有在括号中的模式不匹配时, 才匹配后面的表达式 |
| <code> ?(id &#124; name)ypat &#124; npat) </code> | 检查id或name标识的正则表达式是否存在, 如果存在, 匹配ypat, 否则匹配npat |

**引用组名(`quote`)的上下文环境有哪些? 如何引用组名?**

> `(?P<quote>['"]).*?(?P=quote)`  
1). 在自身模式中引用:  
    * `(?P=quote)`
    * `\1`
2). 在处理匹配项对象m时引用:  
    * `m.group('quote')`
    * `m.end('quote')`  
    * `m.start('quote')`
    * `m.span('quote')`
3). 在函数`re.sub(pat,repl,str)`的参数repl
    * `\g<quote>`
    * `\g<1>`
    * `\1`

**界定符**都是零宽字符, 只表示位置, 没有内容;  
`^$`: 表示字符串的开始位置和结束位置(`\n`所在的位置是结束的位置, 也是开始的位置);  
`(?=)`, `(?!=)`,`(?<=)`, `(?<!)`,表示目标之前和之后的环境;  


In [2]:
m = re.search('(?<=abc)def', 'abcdef')
m2 = re.search(r'(?<=-)\w+', 'spam-egg')
m3 = re.search(r'\w+(?=-)', 'spam-egg')
%C m.group();m2.group(); m3.group()

m.group()  m2.group()  m3.group()
---------  ----------  ----------
'def'      'egg'       'spam'    



**正则表达式中的转义字符**:
>|字符|描述|
|:---|:---|
| `\i` | 使用组编号进行子模式引用 |
| \A | 仅匹配字符串的开始标志 |
| \b | 匹配单词开始和结束时的空白字符串, 单词有单词字符组成, 以空格或其他标点字符结束 |
| \B | 匹配不在单词开始或结尾处的空字符串 |
| \d | 匹配任何十进制数字, r'[0-9] |
| \D | 匹配任何非数字字符, r'[^0-9]' |
| \s | 匹配任何空白字符, r'[\t\n\r\f\v]' |
| \S | 匹配任何非空白字符, r'[^ \t\n\r\f\v]' |
| \w | 匹配任何单词字符 |
| \W | 匹配任何标点字符(单词字符的补集) |
| \z | 仅匹配字符串的结束标志 |
| `\\` | 匹配反斜杠本身 |

**re模块中的匹配规则**(flags):
>| 标志 | 描述 |
|------|------|
| **A, ACSII** |  执行仅8位的ASCII字符匹配 |
| **I, IGNORECASE** |  执行不区分大小写的匹配 |
| **L** |  `\w,\W,\b,\B`与使用地区有关 |
| **M, MULTILINE** | 模式匹配将在所有行中执行 |
| **S, DOTALL** |  让`.`也可以匹配换行符 |
| **X, VERBOSE** |  忽略模式中未转义的空白和注释 |

### re函数

**re模块中的核心函数** :
> 1. **compile**(string,flags):  
    * 将模式字符串编译成正则表达式对象;   
    * 正则表达式对象可以做其他re函数的参数;
2. **escape**(string):  
    * 使所有非单词字符都带上反斜杠;
3. **findall**(pattern, string, flags):  
    * 以列表的形式返回与pat对应的所有未重叠的匹配项,包括空白匹配("");
    * **如果模式包含分组, 将以元组的形式返回所有子项**; 
    * **finditer**: findall的迭代器版本, 迭代器中元素的类型为`Match对象`;
4. **match**(pattern, string, flags):  
    * 检查str的开头是否出现pat的匹配项;
    * 开头出现的话返回Match对象, 不出现的话返回None;  
    * **search**: 不光在字符串的开头检查是否出现匹配项, 还会在字符串中的所有位置检查是否出现匹配项;
5. **split**(pat,str,maxsplit=0)
    * 以pat的匹配项为分割符, 分割字符串;
    * maxsplit为最大分割次数, 默认情况下执行所有可能的拆分;
6. **sub**(pattern,repl,string,count=0):  
    * 使用新串(repl)替换旧串(pat的匹配项);
    * count是执行替换的最高次数;
    * repl如果是一个字符串, 可使用反向引用(`\6`)或`\g<name>`来引用子模式;  
    * 如果repl是一个函数, 通过它可将匹配项对象转换成一个新串;  
    * **subn**: 以元组的形式返回`替换后的字符串`和`执行替换的次数`;

`re.split()`以模式对应的匹配项为分隔符,分割字符串

In [3]:
print(re.split(r'\W+', 'Words, words, words.'))
print(re.split(r'(\W+)', 'Words, words, words.'))
print(re.split(r'\W+', 'Words, words, words.', 1))
print(re.split('[a-f]+', '0a3B9', flags=re.IGNORECASE))

['Words', 'words', 'words', '']
['Words', ', ', 'words', ', ', 'words', '.', '']
['Words', 'words, words.']
['0', '3', '9']


> * 模式匹配了`.`,按此分割字符串导致出现空白;  
* 模式字符串中含有子模式, 输出列表中含有分隔符;  
* 指定最大分割次数, 分割一次出现两个片段;
* 设立规则, 忽略大小写

In [4]:
# 如果在字符串的开头匹配到了分隔符, 分隔符之前会被分割出空白;
re.split(r'(\W+)', '...words, words...')

['', '...', 'words', ', ', 'words', '...', '']

通过`子模式`替换字符串

In [5]:
# 一般的替换语法
re.sub(r'\sAND\s', ' & ', 'Baked Beans And Spam', flags=re.IGNORECASE)

'Baked Beans & Spam'

In [6]:
# 子模式替换
re.sub(r'def\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*\(\s*\):',
       r'static PyObject*\npy_\1(void)\n{',
       'def myfunc():')

'static PyObject*\npy_myfunc(void)\n{'

> `def\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*\(\s*\):`对`def myfunc():`进行了全匹配;  
可以**只使用子模式进行替换字符串**: 相当于提取全匹配中的一个子项, 插入到一个模板当中; 

In [7]:
# 根据替换规则进行替换
def dashrepl(matchobj):
    if matchobj.group(0) == '-': return ' '
    else: return '-'
re.sub('-{1,2}', dashrepl, 'pro----gram-files')

'pro--gram files'

> 如果repl是函数的话, 其实是定义了一个操作, 使用该操作, 可以将匹配项按某种规则变为一个新的字符串;  
* [>>>]: `nltk.re_show('-{1,2}', 'pro----gram-files')`
* [out]: `pro{--}{--}gram{-}files`
* [zzz]: `将`-`变为' ', 将`--`变为`-``

escape()给所有的标点字符(非单词字符)都无情的砍上一刀;

In [8]:
import string
legal_chars = string.ascii_lowercase + string.digits + '_' + "!#$%&'*+-.^`|~:"
print('[%s]+' % re.escape(legal_chars))

[abcdefghijklmnopqrstuvwxyz0123456789_\!\#\$\%\&\'\*\+\-\.\^\`\|\~\:]+


In [9]:
operators = ['+', '-', '*', '/', '**']
print(' .. '.join(map(re.escape, sorted(operators, reverse=True))))
# print(' .. '.join([ re.escape(i) for i in sorted(operators,reverse=True)]))

\/ .. \- .. \+ .. \*\* .. \*


> map(func,xlist):  
通过`函数func`和`xlist`返回一个新的`ylist`, 两个list之间的元素满足关系: `ylist[i] = func(xlist[i])`;

### 正则表达式对象

模式字符串被compile编译之后的称为**模式对象**会正则表达式对象, re对象;

A. **re对象的属性**:  
1). **r.flag**:   
    编译正则表达式时使用的flags参数, 0表示没有指定标志;  
2). **r.groupindex**:  
    返回一个字典, 将`r(?P<id>)`定义的分组名映射到分组编号;  
3). **r.pattern**:  
    返回模式对象的模式;  
B. **re对象的方法**:  
    **r.findall**(r.finditer), **r.match**(r.search), **r.split**, **r.sub**(r.subn)与普通re函数的功能一样;

In [10]:
pattern = re.compile("d")
print(pattern.search("dog"))     # 在索引0处匹配
print(pattern.search("dog", 1,2))  # 在索引1-2至之间找不到匹配项

<_sre.SRE_Match object; span=(0, 1), match='d'>
None


### 匹配项对象

由`search()`和`match()`返回的对象叫做**匹配项对象**;  
匹配项对象包含关于**分组**和**匹配项位置**的信息;

**m.group(g1,g2,...)**:  
* 返回匹配项对象的匹配项或者`子项`  
* **子项**: 子模式所对应的匹配项叫子项;  

In [11]:
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist")
%C m.group(0); m.group(1); m.group(2); m.group(1,2) 

m2 = re.match(r"(?P<first_name>\w+) (?P<last_name>\w+)", "Malcolm Reynolds")
%C m2.group('first_name'); m2.group('last_name')

  m.group(0)    m.group(1)  m.group(2)      m.group(1,2)   
--------------  ----------  ----------  -------------------
'Isaac Newton'  'Isaac'     'Newton'    ('Isaac', 'Newton')

m2.group('first_name')  m2.group('last_name')
----------------------  ---------------------
'Malcolm'               'Reynolds'           



> 如何引用子项?  
* 缺省参数, 或者参数为0, 返回匹配项整体;
* 使用数字或者组名`1`表示第一个子项, g1也可以表示第一个匹配项;    
* 分组编号或分组名无效时抛出IndexError异常;

如果一个组匹配了多次, 只有最后一次匹配的子项可以被应用;  
If a group matches multiple times, only the last match is accessible;

In [12]:
m = re.match(r"([a-z][0-9])+", "a1b2b3")  # Matches 3 times.
m.group(1)

'b3'

**m.groups(default)**:
* 返回一个元组, 其中包含了所有分组对应的子项;
* 如果某分组没有匹配到子项(正好匹配了0次), 默认设置其子项为None;

In [13]:
m = re.match(r"(\d+)\.(\d+)", "24.1632")
m2 = re.match(r"(\d+)\.?(\d+)?", "24")
%C 5 m.groups(); m2.groups(); m2.groups('0')

  m.groups()       m2.groups()      m2.groups('0')
--------------     ------------     --------------
('24', '1632')     ('24', None)     ('24', '0')   



**m.groupdict**():  
* 返回一个字典, 字典的键是组名, 字典的值是组所对应的子项;

In [14]:
m = re.match(r"(?P<first_name>\w+) (?P<last_name>\w+)", "Malcolm Reynolds") 
m.groupdict()

{'first_name': 'Malcolm', 'last_name': 'Reynolds'}

**m.start(group),m.end(group),m.span(group)**:   
返回子项或者整体项的开始和结束索引, 或者索引跨度; 

In [15]:
email = "tony@tiremove_thisger.net"
m = re.search("remove_this", email)
%C m.start(); m.end(); m.span(); email[:m.start()] + email[m.end():]

m.start()  m.end()  m.span()  email[:m.start()] + email[m.end():]
---------  -------  --------  -----------------------------------
7          18       (7, 18)   'tony@tiger.net'                   



**m.re**: 一个正则表达式对象;  
**m.string**: 传递给match或search的字符串;  
**m.pos, m.endpos**: 传递给search()或match()函数的pos值;  
**m.expand**(template): 使用子项替换掉模板中的组名或者组编号;   

**m.lastindex,m.lastgroup**:    
最后一个捕获组的数字索引或者名称;  
`成功`捕获组的最后一个索引或者名字; 如果没有组`成功匹配`将返回None; 如果组名不存在也返回None;  

### RE HOWTO

source: https://docs.python.org/3/howto/regex.html#regex-howto

#### 普通字符和特殊字符

A. **普通字符**:  
* 只会匹配自己的字符称为普通字符;  
* 所有的单词字符都是普通字符: `a-Z`, `_`, `0-9`;  
* 标点字符也有可能是普通字符: `@`;  

B. **特殊字符**(元字符):  
* 特殊字符都来源于标点字符;  
* ex:`^`表示字符串的开始位..

C. **无宽字符**: 只有位置没有内容的字符;  

**指定可选字符集**;  
使用元字符`[]`指定可选字符集;  
可选字符集中需要注意的字符;  
* `-`: 表示字符连续;  
* `$`: 特殊字符单做普通字符;  
* `^`: 如果`^`是字符集的第一个元素, 表示该字符集将作为字符补集;

**`\` 有两种用途**:  

A. 让普通字符变为特殊字符:  
   * 一般的控制符: `\n`表示换行符(成立条件: python);  
   * 正则表达式中的控制符: `\w`表示单词字符(成立条件: re);  

B. 让特殊字符变为普通字符:   
   * `\$`, 将元字符变为普通字符;  

C. 为了避免出现太多的`/`, 正则表达式必须写成原始字符串的形式`r'...'`

**组合元字符`[abc]*`**的含义?
在字符集中进行可重复性随机抽样, 抽样次数由`*`决定;  

* 抽样次数为0: 匹配空白字符`""`;  
* 抽样次数为1: `a`, `b`, `c`;  
* 抽样次数为2: `aa`, `ab`, `ac`, ...;  
* 抽样次数为3: `aaa`, `abb`, `acb`, `ccc`, ...;

####  正则表达式对象及其方法, match对象及其方法

match和search用于`查看匹配项是否存在`;  

In [16]:
# 正则表达式对象
p = re.compile('[a-z]+') 

# p无法匹配空白
%C 5 p; p.match("") 

m = p.match('tempo')
%C 5 m; m.group(); m.span() 

# match与search的区别
m2 = p.search('::: message')
%C m2; m2.group();m2.span()

                p                     p.match("")
---------------------------------     -----------
re.compile(r'[a-z]+', re.UNICODE)     None       

                         m                              m.group()     m.span()
---------------------------------------------------     ---------     --------
<_sre.SRE_Match object; span=(0, 5), match='tempo'>     'tempo'       (0, 5)  

                          m2                            m2.group()  m2.span()
------------------------------------------------------  ----------  ---------
<_sre.SRE_Match object; span=(4, 11), match='message'>  'message'   (4, 11)  



> * match返回None对象或者匹配项对象;  
* group()返回匹配项;  
* span()回匹配项对象的索引跨度;  
* match()只会在字符串开头查找匹配项, 而search会在所有位置查找;
* m的start只能是0, 而m2的start是第一个匹配项的位置;

findall和finditer用于`列出所有的匹配项或匹配项属性`;   
findall()返回匹配项列表, finditer()返回关于`匹配项对象`的迭代器;  

In [17]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')

['12', '11', '10']

In [18]:
iterator = p.finditer('12 drummers drumming, 11 ... 10 ...')
print("type of finditer(): ",iterator)
for i, match in enumerate(iterator):
    print("`type, span, match(group)`: ", match )

type of finditer():  <callable_iterator object at 0x00000268F3127400>
`type, span, match(group)`:  <_sre.SRE_Match object; span=(0, 2), match='12'>
`type, span, match(group)`:  <_sre.SRE_Match object; span=(22, 24), match='11'>
`type, span, match(group)`:  <_sre.SRE_Match object; span=(29, 31), match='10'>


#### 如何使正则表达式的编写变得更加简单?

指定re的flags参数为`re.VERBOSE`:  
* re模式中的空格将被编译器忽略;  
* 除非该空格位于字符类中或者其前面带有未转义的反斜杠;  
* 因此可以随意的缩进RE, 插入注释;

In [19]:
charref1 = re.compile("&#(0[0-7]+"
                     "|[0-9]+"
                     "|x[0-9a-fA-F]+);")

charref2 = re.compile(r"""
                     &[#]                # Start of a numeric entity reference
                     (
                       0[0-7]+           # Octal form
                       | [0-9]+          # Decimal form
                       | x[0-9a-fA-F]+   # Hexadecimal form
                      )
                     ;                   # Trailing semicolon
                        """, re.VERBOSE)

`redemo.py`: 编写`re`表达式的辅助工具;  
由tkinter编写(Python界面处理模块), 可由Python直接打开;  
https://github.com/python/cpython/blob/3.7/Tools/demo/redemo.py

#### 关于特殊字符的细节

`^`表示原始字符串的开头, 而不是表示目标的开头

In [20]:
# 在原始字符串的开头匹配`Form`
print(re.search('^From', 'From Here to Eternity'))
print(re.search('^From', 'Reciting From Memory'))

<_sre.SRE_Match object; span=(0, 4), match='From'>
None


`$`匹配原始字符串的末尾, `\n`所占的位置才是字符串的末尾

In [21]:
# 原始字符串的末尾不见换行符的话, 原始字符串的末尾就是`$`的位置
print(re.search('}$', '{block}'))  
print(re.search('}$', '{block} '))

<_sre.SRE_Match object; span=(6, 7), match='}'>
None


In [22]:
# 原始字符串的末尾可见换行符的话, `\n`所占的位置就是`$`的位置(`$`的位置不是`\n`之后或者`\n`之前的那一位)
print(re.search('}$', '{block}\n')) 
print(re.search('}$', '{block}\n ')) 
print(re.search('^X', 'A\nB\nX', re.MULTILINE))

<_sre.SRE_Match object; span=(6, 7), match='}'>
None
<_sre.SRE_Match object; span=(4, 5), match='X'>


> 原始字符串的末尾如果没有换行符, `re`会认为其末尾就是换行符, 而`$`永远表示换行符(`\n`)所占的那一位;  
`^`也代表换行符所占的那一位么;

设置目标所处的前后环境: `\b`;  
* `\b`是单词与单词的边界, 是一个**零宽界定符**(zero-width-assertion), 只用于设置目标所处的环境, 不能指代任何字符, 常常用于分词;  
* `\W`也可用于分词, 但是`\W`可以明确的指代一个标点, 使得分词常常带着标点;  

In [23]:
# class的两侧不得出现单词字符
p = re.compile(r'\bclass\b')
print(p.search('no class at all'))

<_sre.SRE_Match object; span=(3, 8), match='class'>


In [24]:
# class左侧没有空白字符, 所以没有匹配项
print(p.search('the declassified algorithm'))

None


In [25]:
# class两侧有标点字符可以进行匹配->`\b`是单词与单词之间的边界空白符, 不是单词字符与标点字符之间的空白边界符
# 通过`\b`进行分词不会带标点
print(p.search('one **class** is'))

<_sre.SRE_Match object; span=(6, 11), match='class'>


In [26]:
# 通过`\W*`进行分词会带标点
p2 = re.compile(r'\W*class\W')
print(p2.search('one **class** is'))

<_sre.SRE_Match object; span=(3, 12), match=' **class*'>


> 使用`\b`和`\W`分词时的区别?
* `\b`是一个界定符, 表示单词字符之间的边界, 其字面量是空白的;  
* `\b`只是设置匹配项的环境规则, 不可能匹配到字符; 
* `\W`是一个有字面量的标点字符, 会将与其对应的字符反映到匹配项中;

In [27]:
# 不在原始字符串中的`\b`表示Python中的退格符
p2 = re.compile('\bclass\b')
print(p2.search('no class at all'))
p3 = re.compile('\\bclass\\b')
print(p3.search('no class at all'))

None
<_sre.SRE_Match object; span=(3, 8), match='class'>


#### 如何在原始字符串中解析不同的兴趣点?

* 对re进行分组处理, 让每个分组分别对应于不同的兴趣点;    
* 进行分组解析的前提是`使用正则表达式完整的匹配原始字符串`;

使用正则表达式完整的匹配原始字符串

In [28]:
p = re.compile('(ab)*')
print(p.match('ababababab').span())

(0, 10)


>* `()*`的含义: 零次或多次重复`ab`;  
* `(ab)*`完整的匹配了'ababababab', 所以其跨度等于字符串的长度;

**子模式的匹配项对象**也有group(),groups(), span()等方法;  
接收**组名**作为参数, 返回子项的属性;  
不接收组名作为参数, 返回匹配项整体的属性;

In [29]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
%C m.group(0); m.group(1); m.group(2); m.group(2,1,2); m.groups()

m.group(0)  m.group(1)  m.group(2)    m.group(2,1,2)    m.groups() 
----------  ----------  ----------  -----------------  ------------
'abcd'      'abc'       'b'         ('b', 'abc', 'b')  ('abc', 'b')



> * 左括号`(`的`第次`就的组所对应的编号;
* group(0): 整个模式的匹配项, 匹配项整体;    
* group(2,1): group可以一次接收多个组号;  
* groups(): 将`所有组的匹配项`合并为一个元组, 将所有子项打包成一个元组;

模式的反向引用: **`\1`不是模式的复制, 而是对子项的复制**;

In [30]:
p1 = re.compile(r'\b(\w+)\s+\1\b')
p2 = re.compile(r'\b(\w+)\s+(\w+)\b')
print(p1.search('Paris in the the spring').group(0))
print(p2.search('Paris in the the spring').group(0))

the the
Paris in


> * 反向引用时必须使用原始字符串;  
* 反向引用中, 引用的是模式的匹配项, 而不是模式本身;

`(?:...)`: 定义一个**无捕获组**, 只用于匹配`()`内的模式, 但是不对其进行引用;   
`(?:...)`对于修改已有组尤其有用, 因为你可以不用改变所有其他组号的情况下添加一个新组;

In [31]:
m = re.match("([abc])+", "abcd")
m2 = re.match("([abc])+?", "abcd")
m_nc = re.match("(?:[abc])+", "abc")
m2_nc = re.match("(?:[abc])+?", "abcd")

%C m.groups(); m2.groups();; m_nc.groups(); m2_nc.groups()

m.groups()  m2.groups()
----------  -----------
('c',)      ('a',)     

m_nc.groups()  m2_nc.groups()
-------------  --------------
()             ()            



> * `([abc])+`相比于`[abc]+`只是添加了一层分组, 而并没有改变`[abc]+`可随机重复取样的特性;   
* `+`的贪婪模式, 会导致模式的匹配项为`abc`, 但是与分组对应的匹配项应该是哪个字母呢? **与分组对应的匹配项是最后一个字母**;  
    * `+`=1时,取`([])`为a(才可以与原始字符串匹配,贪婪);  
    * `+`=2时,取`([])`为b(才可以与原始字符串匹配,贪婪);
    * `+`=3时,取`([])`为c(贪婪); 但是此时匹配过程必须终止, 因为原始字符串中没有可匹配的字符了; 此时子模式`()`对应的最后一个字母是c;  
* `+?`是非贪婪模式, `([abc])+`会尽可能少的匹配字符, 只匹配一个字符最少, 就是`a`;

>2. `(?:[abc])+`和`(?:[abc])+?`是一个非捕获组, 只设置总匹配项被匹配的条件, 但是忽略子匹配项的输出;

`(?P<name>...)`: 定义一个**命名组**, 使用`(?P=name)`可对命名组的进行逆向引用;  
`(\b\w+)\s+\1`==`(?P<word>\b\w+)\s+(?P=word)`

In [32]:
p = re.compile(r'(?P<word>\b\w+\b)')
p2 = re.compile(r'(?P<word>\W*\w+\W*)')

m = p.search('(((( Lots of punctuation )))')
m2 = p2.search('(((( Lots of punctuation )))')
%C 5 m.group('word'); m.groups(); m.groupdict();; m2.group('word'); m2.groups();m2.groupdict()

m.group('word')     m.groups()      m.groupdict()  
---------------     ----------     ----------------
'Lots'              ('Lots',)      {'word': 'Lots'}

m2.group('word')       m2.groups()           m2.groupdict()    
----------------     ---------------     ----------------------
'(((( Lots '         ('(((( Lots ',)     {'word': '(((( Lots '}



**前向界定符**,也是零宽定界符, 包括**前项肯定定界符**`(?=...)`和**前项否定定界符**`(?!..)`;  

匹配所有扩展名为`.bat`的文件, 但不能匹配`.bat*`的文件?

In [33]:
pat = r'.*\.(?!bat$).*$'
strings = ['acbr.bat', 'babr.bat*', '@br.exe', '?.txt', 'br.br', '02 br.clc' ]
for string in strings:
    f = re.findall(pat,string)
    print(f)

[]
['babr.bat*']
['@br.exe']
['?.txt']
['br.br']
['02 br.clc']


**`.*\.(?!bat$).*$`**:

   * `.*`匹配任意多个单词字符或者标点字符的组合;  
   * `\.`: 匹配一个圆点符号, 等于`[.]`;  
   * `(?!bat)`: 前项否定定界符, 只有如今字符不是`bat`时才会继续匹配其他字符;   
   * `(?!bat$)`: 加上`$`之后表示, 该处一直到行尾不是`bat`的话, 才会继续寻找匹配字符; 从而使的`.batch`会得到匹配, 而`.bat`不会得到匹配;  
   * `.*$`: 继续寻找匹配字符, 直到行尾;  
   * `$`: 结束寻找, 如果找到模式的所有匹配字符, 就宣称找到了匹配项; 如果某一字符找不到, 就在该字符处终止寻找, 宣称没有匹配项;  

**描述匹配项的寻找过程**:  
根据模式的字符顺序, 一一与字符串中的字符进行对比;  
如果**模式字符**与**原始字符串字符**对应, 则接着判断下一个**模式字符**是否与**原始字符**对应;  
直到某个模式字符与字符串字符无法对应, 则宣称**找不到匹配项**;  
或者模式字符与字符串的最后一个末尾符对应, 则宣称**找到匹配项**; 

**split()的模式参数中, 如果含有子模式** 

In [34]:
p = re.compile(r'\W+')
p2 = re.compile(r'(\W+)')
print(p.split('This... is a test.'))
print(p2.split('This... is a test.'))

['This', 'is', 'a', 'test', '']
['This', '... ', 'is', ' ', 'a', ' ', 'test', '.', '']


> maxsplit是分割符的个数, 而不是分割之后碎片的个数;  
如果**捕获括号, 出现在split函数的模式参数中**, 分割符会随着分割片段一并输出;

#### 字符串查找与替换的re实现(Ctrl+F,Ctrl+H)

In [35]:
# 用单词 "colour" 替换颜色名
p = re.compile( '(blue|white|red)')
p.sub( 'colour', 'blue socks and red shoes')

'colour socks and colour shoes'

In [36]:
# subn() 方法作用一样，但返回的是包含新字符串和替换执行次数的两元组
p = re.compile( '(blue|white|red)')
print(p.subn( 'colour', 'blue socks and red shoes'))
print(p.subn( 'colour', 'no colours at all'))

('colour socks and colour shoes', 2)
('no colours at all', 0)


> subn()与sub的作用一样, 只不过subn会同时返回执行替换的次数;

findall能否匹配**空白字符**

In [37]:
p = re.compile('x?')
p2 = re.compile('x??')

%C 5 re.findall(p, 'abxd');re.findall(p2, 'abxd'); re.subn(p,'-','abxd')
nltk.re_show('x?','abxd')

re.findall(p, 'abxd')     re.findall(p2, 'abxd')     re.subn(p,'-','abxd')
---------------------     ----------------------     ---------------------
['', '', 'x', '', '']     ['', '', '', '', '']       ('-a-b-d-', 4)       

{}a{}b{x}d{}


> 包括空白字符在内, findall总共发现5个匹配项?   
* `re.findall(p, 'abxd')`只会忽略匹配项之前的空白字符(贪婪,匹配了空白和x,只显示x), `{1}a{2}b{x,3}{4}d{5}`;  
* `re.findall(p2, 'abxd')`会忽略x而不会忽略空白, `{1}a{2}b{3}x{4}d{5}` ;
* 空白项只有在它没有紧挨着前一个匹配项时才会被替换掉: `{-}a{-}b{x:-}{X}d{-}`;   
* `re_show()`和`re.subn()`会忽略匹配项前后的两个空白字符;  

In [38]:
%C re.findall(r'.*', 'ABCD'); re.findall(r'.*$', 'ABCD'); re.findall(r'^.*', 'ABCD')

re.findall(r'.*', 'ABCD')  re.findall(r'.*$', 'ABCD')  re.findall(r'^.*', 'ABCD')
-------------------------  --------------------------  --------------------------
['ABCD', '']               ['ABCD', '']                ['ABCD']                  



> * findall本身就会忽略匹配项之前的空白字符, 所以`^`和匹配项之间不会出现空白字符, 匹配项之后可以匹配空白字符; 
* 设置`^`之后, 说明匹配项只能出现在字符串的开头, 所以不再匹配匹配项之后的空白字符;  
* 不管`$`字符有木有出现在模式的结尾, 匹配项和`$`之间肯定会出现空白字符;  

将字符串中的`section`转换为`subsection`

In [39]:
p = re.compile('section{ ( [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\1}','section{First} section{second}')

'subsection{First} subsection{second}'

In [40]:
p = re.compile('section{ (?P<name> [^}]* ) }', re.VERBOSE)

In [41]:
p.sub(r'subsection{\1}','section{First}')

'subsection{First}'

In [42]:
p.sub(r'subsection{\g<1>}','section{First}')

'subsection{First}'

In [43]:
p.sub(r'subsection{\g<name>}','section{First}')

'subsection{First}'

使用**替换函数repl_func**将十进制翻译成十六进制

In [44]:
def hexrepl( match ):
    "Return the hex string for a decimal number"
    value = int( match.group() )
    return hex(value)
p = re.compile(r'\d+')
p.sub( hexrepl, 'Call 65490 for printing, 49152 for user code.') 

'Call 0xffd2 for printing, 0xc000 for user code.'

In [45]:
for match in re.finditer( r'\d+', 'Call 65490 ... 49152 ...'):   
    print("{}: {} ".format(match.group(), hexrepl(match)))

65490: 0xffd2 
49152: 0xc000 


> 当参数`repl`是一个函数时, sub的内部进行了什么样的操作?  
* 调用finditer(), 返回所有的模式项对象;  
* 将模式项对象分别作为repl()的参数, 计算一个新的字符串;  
* 用新的字符串替换相应原始对象中的旧字符串;

#### 除了RE有没有更好的选择

在使用 re 模块之前, 先考虑一下你的问题是否可以用更快,更简单的字符串方法来解决; 
例如, 将文本中的`begins`替换成 `starts`, 不需要使用复杂的正则表达式:  

In [46]:
s = 'Night gathers, and now my watch begins.'
s.replace('begins', 'starts')

'Night gathers, and now my watch starts.'

> 从一个字符串中删除单个字符或用另一个字符来替代它, 使用translate()能够简单的实现这两个任务;

#### 正则表达式的贪婪模式和懒惰模式

In [47]:
s = '<html><head><title>Title</title>'
%C len(s); re.match('<.*>', s).span();re.match('<.*?>', s).span();; re.match('<.*>', s).group(); re.match('<.*?>', s).group()

len(s)  re.match('<.*>', s).span()  re.match('<.*?>', s).span()
------  --------------------------  ---------------------------
32      (0, 32)                     (0, 6)                     

   re.match('<.*>', s).group()      re.match('<.*?>', s).group()
----------------------------------  ----------------------------
'<html><head><title>Title</title>'  '<html>'                    



### RE Examples

#### 查找对牌

In [48]:
def displaymatch(match):
    if match is None:
        return None
    return '<Match: %r, groups=%r>' % (match.group(), match.groups())

In [49]:
# 查看扑克牌是否有效
valid = re.compile(r"^[a2-9tjqk]{5}$")
print(displaymatch(valid.match("akt5q"))) #有效
print(displaymatch(valid.match("akt5e"))) #无效
print(displaymatch(valid.match("727ak"))) #有效-对7

<Match: 'akt5q', groups=()>
None
<Match: '727ak', groups=()>


In [50]:
#查看是否存在对牌
pair = re.compile(r".*(.).*\1") 
print(displaymatch(pair.match("717ak"))) #有对
print(displaymatch(pair.match("718ak"))) # 无对
print(displaymatch(pair.match("354aa"))) #有对

<Match: '717', groups=('7',)>
None
<Match: '354aa', groups=('a',)>


<font color=red size=6> $$1 </font>

#### 模拟 scanf()

> python中没有C中的`scanf("%d %d",&a,&b)`;  
但是使用RE可以在Python中模拟出scanf()的功能:

> | scanf() | RE |
|:--------|:-----------------------------|
| scanf() Token | Regular Expression |
| %c | . |
| %5c | .{5} |
| %d | [-+]?\d+ |
| %e, %E, %f, %g | [-+]?(\d+(\.\d*)? &#124; \.\d+)([eE][-+]?\d+)? |
| %i | [-+]?(0[xX][\dA-Fa-f]+ &#124; 0[0-7]* &#124; \d+) |
| %o | [-+]?[0-7]+ |
| %s | \S+ |
| %u | \d+ |
| %x, %X | [-+]?(0[xX])?[\dA-Fa-f]+ |

#### 制作电话簿

In [51]:
text = """Ross McFluff: 834.345.1254 155 Elm Street

Ronald Heathmore: 892.345.3428 436 Finley Avenue
Frank Burger: 925.541.7625 662 South Dogwood Way


Heather Albrecht: 548.326.4584 919 Park Place"""
print(text)

Ross McFluff: 834.345.1254 155 Elm Street

Ronald Heathmore: 892.345.3428 436 Finley Avenue
Frank Burger: 925.541.7625 662 South Dogwood Way


Heather Albrecht: 548.326.4584 919 Park Place


In [52]:
# 以换行符为分隔符对text进行分割
entries = re.split("\n+", text)
entries

['Ross McFluff: 834.345.1254 155 Elm Street',
 'Ronald Heathmore: 892.345.3428 436 Finley Avenue',
 'Frank Burger: 925.541.7625 662 South Dogwood Way',
 'Heather Albrecht: 548.326.4584 919 Park Place']

In [53]:
# 以` `或`: `为分割符分割每个字符串
# first-name, last-name, tel-number, house-number,  street
[re.split(":? ", entry, 4) for entry in entries]

[['Ross', 'McFluff', '834.345.1254', '155', 'Elm Street'],
 ['Ronald', 'Heathmore', '892.345.3428', '436', 'Finley Avenue'],
 ['Frank', 'Burger', '925.541.7625', '662', 'South Dogwood Way'],
 ['Heather', 'Albrecht', '548.326.4584', '919', 'Park Place']]

#### 文本改写

除了收尾字母以外, 将字符串中所有字母的顺序随意打乱;

In [54]:
import numpy as np
def repl(m):
    inner_word = list(m.group(2))
    np.random.shuffle(inner_word)
    return m.group(1) + "".join(inner_word) + m.group(3)
text = "Professor Abdolmalek, please report your absences promptly."
re.sub(r"(\w)(\w+)(\w)", repl, text)

'Posrsofer Aoebadlmlk, pslaee rperot your acbneess prltpomy.'

#### 查找副词及其位置

In [55]:
text = "He was carefully disguised but captured quickly by police."
for m in re.finditer(r"\w+ly", text):
    print('{:02d}-{:02d}: {:s}'.format( m.start(),m.end(),m.group(0)))

07-16: carefully
40-47: quickly


#### 编写分词器

**namedtuple**():  
* 创建一个自定义的`tuple对象`, 并且规定了tuple中元素的个数, 并且可以通过属性而不是索引来引用tuple中的某个元素;  
* 定义一种新的数据类型, 具备tuple的不变性, 并且可以根据属性来引用元素;

In [56]:
import collections
Point = collections.namedtuple('Point', ['x', 'y'])
p = Point(1, 2)
%C 5 p.x; p.y

p.x     p.y
---     ---
1       2  



通过RE来指定文本类型, 首先将它们放进一个正则表达式, 然后不间断的匹配它们;

In [57]:
Token = collections.namedtuple('Token', ['typ', 'value', 'line', 'column'])
def tokenize(code):
    # 
    keywords = {'IF', 'THEN', 'ENDIF', 'FOR', 'NEXT', 'GOSUB', 'RETURN'}
    # token格式
    token_specification = [
        ('NUMBER',  r'\d+(\.\d*)?'),  # 整数或十进制数
        ('ASSIGN',  r':='),           # 匹配操作符
        ('END',     r';'),            # 声明终止符
        ('ID',      r'[A-Za-z]+'),    # 标识符
        ('OP',      r'[+\-*/]'),      # 算数运算符
        ('NEWLINE', r'\n'),           # 行尾
        ('SKIP',    r'[ \t]+'),       # 跳转空格或者制表符
        ('MISMATCH',r'.'),            # 其他字符
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    line_num = 1
    line_start = 0
    for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group(kind)
        if kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
        elif kind == 'SKIP':
            pass
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')
        else:
            if kind == 'ID' and value in keywords:
                kind = value
            column = mo.start() - line_start
            yield Token(kind, value, line_num, column)

statements = '''
    IF quantity THEN
        total := total + price * quantity;
        tax := price * 0.05;
    ENDIF;
'''

for token in tokenize(statements):
    print(token)

Token(typ='IF', value='IF', line=2, column=4)
Token(typ='ID', value='quantity', line=2, column=7)
Token(typ='THEN', value='THEN', line=2, column=16)
Token(typ='ID', value='total', line=3, column=8)
Token(typ='ASSIGN', value=':=', line=3, column=14)
Token(typ='ID', value='total', line=3, column=17)
Token(typ='OP', value='+', line=3, column=23)
Token(typ='ID', value='price', line=3, column=25)
Token(typ='OP', value='*', line=3, column=31)
Token(typ='ID', value='quantity', line=3, column=33)
Token(typ='END', value=';', line=3, column=41)
Token(typ='ID', value='tax', line=4, column=8)
Token(typ='ASSIGN', value=':=', line=4, column=12)
Token(typ='ID', value='price', line=4, column=15)
Token(typ='OP', value='*', line=4, column=21)
Token(typ='NUMBER', value='0.05', line=4, column=23)
Token(typ='END', value=';', line=4, column=27)
Token(typ='ENDIF', value='ENDIF', line=5, column=4)
Token(typ='END', value=';', line=5, column=9)


<font color=red size=6> $$tail </font>